In [4]:
import requests
import pandas as pd
import time

# Load titles from CSV (expects a column named "series")
df = pd.read_csv("just_titles.csv")
titles = df["series"].tolist()

# Initialize results list
results = []

# Define batch size and output file
batch_size = 10
output_file = "related_entries_results.csv"

# Public Jikan REST API
API_URL = "https://api.jikan.moe/v4/anime"

def search_title(title):
    """Return the first matched anime with ID and basic info."""
    try:
        response = requests.get(f"{API_URL}", params={"q": title, "limit": 1})
        if response.status_code == 200:
            data = response.json()
            if data["data"]:
                first = data["data"][0]
                return {
                    "search_title": title,
                    "id": first["mal_id"],
                    "type": "anime",
                    "related_series": [],
                    "format": first.get("type"),
                    "start_date": first.get("aired", {}).get("from"),
                    "end_date": first.get("aired", {}).get("to"),
                    "favorites": first.get("favorites"),
                    "popularity": first.get("popularity"),
                    "score": first.get("score")
                }
    except Exception as e:
        print(f"Error searching for title '{title}': {e}")
    return None

def get_related_entries(title_info):
    """Fetch related series. Jikan doesn't provide full metadata here."""
    related_entries = []
    try:
        response = requests.get(f"{API_URL}/{title_info['id']}/relations")
        if response.status_code == 200:
            relations_data = response.json()["data"]
            for rel in relations_data:
                for entry in rel.get("entry", []):
                    related_entries.append({
                        "search_title": title_info["search_title"],
                        "relation_type": rel.get("relation"),
                        "related_title": entry.get("name"),
                        "related_type": entry.get("type"),
                        "related_mal_id": entry.get("mal_id")
                    })
    except Exception as e:
        print(f"Error fetching related entries for '{title_info['search_title']}': {e}")
    return related_entries

# Main loop to process titles
for i, title in enumerate(titles):
    print(f"\n[{i+1}/{len(titles)}] Searching: {title}")
    
    # Fetch main title info
    main_info = search_title(title)
    
    if main_info:
        # Fetch related entries
        entries = get_related_entries(main_info)
        results.extend(entries)
    else:
        print(f"  No match found for '{title}'")
    
    # Save progress after each batch
    if (i + 1) % batch_size == 0 or i == len(titles) - 1:
        pd.DataFrame(results).to_csv(output_file, index=False)
        print(f"Saved progress at entry {i+1}")
    
    # Rate limiting
    time.sleep(1)

print("\n✅ All done!")
